Here’s a **detailed, structured set of notes** from your transcript on **DBMS – Backup & Recovery/4: Recovery/3** with both the *learning outcomes* covered in depth.

---

# **DBMS – Backup & Recovery/4: Recovery/3**

## **1. Recovery with Early Lock Release**

### **1.1 Context**

* **Earlier focus**: Recovery at the transaction level (undo/redo of read/write operations).
* **Reality**: At the physical storage level (disk), recovery also involves:

  * Data files
  * Index files (e.g., B+ Trees)
  * Other system data structures
* These structures require their own concurrency control and recovery mechanisms.

---

### **1.2 The Situation**

* **Write operation**:

  * Data written in memory (buffer).
  * Buffer Manager eventually flushes the block to disk.
* **Concurrency issues**:

  * Multiple buffers can be written concurrently.
  * For **indices (B+ Trees)**, concurrency control algorithms allow:

    * Concurrent read/write
    * Shared/exclusive locks
  * To increase throughput, **locks may be released early** (before transaction commit).

---

### **1.3 Early Lock Release**

* **Definition**: Releasing a low-level lock on a resource (e.g., a B+ Tree node) before the transaction completes.
* **Reason**:

  * B+ Tree is a **centralized resource** accessed by many transactions.
  * Holding locks until transaction commit (2-phase locking) → throughput drops.
  * Early release lets other transactions update the same node sooner.
* **How it works**:

  * Lock is held only for the duration of updating the node.
  * Released immediately after the update, not tied to transaction commit.

---

### **1.4 Example Problem**

* **T1** inserts `(V1, R1)` into a B+ Tree node.
* Lock is released **before T1 completes**.
* **T2** inserts `(V2, R2)` into the same node **before T1 commits**.
* Consequences:

  * Node contents and structure may change (splits, shifts).
  * Position of `(V1, R1)` may move before T1 finishes.
  * If T1 is rolled back:

    * A *physical undo* (restore old node value) would also remove `(V2, R2)` from T2 — **wrong**, because T2 might have committed.
    * This creates a new inconsistent state.

---

### **1.5 Logical Undo**

* **Solution**: Instead of restoring old node physically, undo by a **logical operation**.
* **Concept**:

  * Undo insert by performing a delete.
  * Undo delete by performing an insert.
* **Terminology**:

  * **Logical Undo Logging**: Records the *operation* needed to logically reverse the original action.
  * **Physical Undo Logging**: Records the *old value* to restore the exact prior state.
* **Key Points**:

  * Redo is still **physical** (simpler and efficient).
  * Undo for early lock release cases is **logical**.

---

### **1.6 Logging Structure with Logical Undo**

* For an operation in transaction **Ti**:

  1. **Operation Begin**:

     * Log: `(Ti, Oj, Operation Begin)`

       * `Oj` = Operation ID.
  2. **Physical Update Log Records**:

     * Store **old value** (for physical undo if needed before op completes).
     * Store **new value** (for physical redo).
  3. **Operation End**:

     * Log: `(Ti, Oj, Operation End, Logical Undo Info)`

       * Logical undo info: Counter-operation (e.g., delete for insert).
* **Crash Cases**:

  * **Before Operation Ends**:

    * Use physical undo.
  * **After Operation Ends**:

    * Use logical undo (ignore physical undo info).

---

### **1.7 Example**

* T1 inserts `(K5, RID7)` into index `I9`:

  * **Operation Begin**: `(T1, O1, Begin)`
  * **Physical Updates**:

    * Change `X: 10 → K5`
    * Change `Y: 45 → RID7`
  * **Operation End**:

    * Logical Undo: `Delete from I9 key=K5, RID=RID7`

---

### **1.8 Rollback with Logical Undo**

* **Steps**:

  1. Scan log backwards.
  2. If an **Operation End** with undo info is found:

     * Perform **logical undo**.
     * Log rollback action (like normal operation).
     * End rollback with **Operation Abort**.
  3. Skip all earlier records for that operation until **Operation Begin**.
  4. If **Operation Abort** is found during scan:

     * Skip to corresponding **Operation Begin**.
  5. At **Transaction Start**, log **Transaction Abort**.

---

### **1.9 Rollback Example**

* T0 updates C: `700 → 600` (O1, undo = C+100)
* T1 updates C: `600 → 400` (O2, undo = C+200)
* T0 aborts:

  * Logical undo O1: C = 500
  * B (unrelated var) undone physically.
  * Log: Operation Abort for O1, T0 Abort.
* Final:

  * T1 commits with C = 500 (effect of its update remains).

---

### **1.10 Failure Recovery with Logical Undo**

* Redo Phase:

  * Identify committed transactions.
  * Build undo list (incomplete transactions).
* Undo Phase:

  * Apply logical undo for completed operations with early lock release.
  * Apply physical undo for incomplete operations without early lock release.

---

## **2. Planning for Backup and Recovery**

### **2.1 Key Factors to Consider**

1. **Importance of Data**:

   * Critical data needs more frequent backups.
   * Non-critical data can have less frequent backups.
2. **Frequency of Change**:

   * Highly volatile data → frequent/daily backups (e.g., transaction logs).
   * Static data → occasional backups (e.g., account holder info).
3. **Speed of Recovery Needed**:

   * Is downtime acceptable?
   * Faster recovery requires more resources and cost.

---

### **2.2 Cost Factors**

* **Hardware/Software**:

  * Backup servers, storage media, automated backup systems.
* **Staff**:

  * Skilled personnel for quick recovery operations.
  * Higher staff costs for high-speed recovery capability.

---

### **2.3 Storage Decisions**

* **On-site** vs **Off-site**:

  * On-site: Quick recovery, but vulnerable to site disasters.
  * Off-site: Safer from local disasters, slower to restore.
* **Multiple Copies**:

  * Improves reliability, but increases cost.

---

### **2.4 Strategy Takeaways**

* Not all data needs the same backup frequency or method.
* Balance between **cost**, **speed**, and **data criticality**.
* Backup/recovery planning is usually a **senior-level decision**, but DB engineers must understand the reasoning.

---

## **3. Learning Outcomes Recap**

✅ **Recovery with Early Lock Release**:

* Why early lock release is used.
* Problem it creates for physical undo.
* Logical undo as a solution.
* Logging structure with operation begin/end and undo info.
* Rollback process with logical undo.
* Examples and crash recovery steps.

✅ **Planning for Backup and Recovery**:

* Factors affecting backup/recovery planning.
* Cost and resource considerations.
* On-site/off-site/multiple copy strategies.
* Role of data importance and change frequency in scheduling backups.